### Data collection for various metrics:

For Ours:
Data Collection for Safety and Stability Metrics (Ignore the efficiency metrics even though they are displayed). 

Collection performed in 2 stages

    A. Data collection for Safety and CAV (Controller Acceleration Variation) 
    B. Data collevtion for WAR (Wave Attenuation Ratio)

### Notes:

- Desired velocity change for FS May need monitoring (May not be set for all possible lengths)
- PI can have failures, so have to turn ON render and manually check
- Generated data will be saved in the folder `test_time_rollout`

- At test time: The inflow should be fixed and be same as the classic controllers 
- Sim step: At training, was set to 0.5. At test, set to 0.1

## When analyzing the csv files
- For RL shocks start at 7400 and end at 11000
- To make it fair, for classic controller the rollouts goes upto 11000 with shocks that come up at 7400 (for some reason, the warmup value makes the horizon longer but does not push the shock start time back.)


### Additional Notes
- When sampling shock vehicles, sometimes a vehicle that has exited the network may be sampled and cause an error. Redo the experiment in that case.

In [1]:
NUM_ROLLOUTS = 5 # There is more variation in the sampling


# Just change the av_frac and rerun the experiments
AV_FRAC =  0.05 #0.05, 0.2, 0.4, 0.6

### 1. (100%) IDM 

In [ ]:
# idm (although av_frac is not used, must be supplied)
!python classic.py --method idm --gen_emission --num_rollouts $NUM_ROLLOUTS\
--render --shock --av_frac 1.0 --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method idm --start_time 500 --end_time 1220 --sim_step 0.5

### 2. Multiple Vehicle Control at % deployments

### FS

In [ ]:
!python classic.py --method fs --gen_emission --num_rollouts $NUM_ROLLOUTS \
--render --shock --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method fs --start_time 500 --end_time 1220 --sim_step 0.5

### PIwS

In [ ]:
# piws
!python classic.py --method piws --gen_emission --num_rollouts $NUM_ROLLOUTS\
--shock --render --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method pi --start_time 500 --end_time 1220 --sim_step 0.5

### BCM

In [ ]:
# bcm 
!python classic.py --method bcm --num_rollouts $NUM_ROLLOUTS \
--gen_emission --shock --render --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method bcm --start_time 500 --end_time 1220 --sim_step 0.5

### LACC

In [ ]:
# lacc
!python classic.py --method lacc --num_rollouts $NUM_ROLLOUTS\
--gen_emission --shock --render --av_frac $AV_FRAC --sim_step 0.5 --shock_start_time 500 \
--shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method lacc --start_time 500 --end_time 1220 --sim_step 0.5

### Notes: 
- Some things are different for Vinitsky and Ours
- Sim steps is 0.5 so although horizon is 11000. It ends at 5500. These are longer than what is seen at training (1750).
- Shocks normally begin at 7400 and end at 11000 (for 0.1 timesteps = 360 seconds as sim_step) but here (at 0.5 sim_steps):

1. begin at 500 and end at 1240 i.e., they run for 720 steps (360 seconds)

- Lokks like not a single change in eval_metrics.
- There is no warmup at test time (same for classic controllers)

### Vinitsky
- Now we have different policies for each av_frac
- Always monitor, can unexpectedly quit

### (5%)

In [ ]:
# For some reason, this replaces its own file so have to be done this way
# First change the avfrac in the exp_configs 
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Vinitsky_et_al/Trained_policies/5_percent/PPO_BottleneckDesiredVelocityEnv-v0_3057f496_2024-02-21_16-50-124wlos7bj 130\
    --method vinitsky --num_rollouts 1 --gen_emission --shock --render --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220
    

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (20%)

In [ ]:
# For some reason, this replaces its own file so have to be done this way
# First change the avfrac in the exp_configs 
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Vinitsky_et_al/Trained_policies/20_percent/PPO_BottleneckDesiredVelocityEnv-v0_f4771a48_2024-02-22_09-23-310tvp2di2 252\
    --method vinitsky --num_rollouts 1 --gen_emission --shock --render --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220
    

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (40%)

In [ ]:
# For some reason, this replaces its own file so have to be done this way
# First change the avfrac in the exp_configs 
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Vinitsky_et_al/Trained_policies/40_percent/PPO_BottleneckDesiredVelocityEnv-v0_8a4c5b66_2024-02-23_13-51-23_yzy25bs 502\
    --method vinitsky --num_rollouts 1 --gen_emission --shock --render --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (60%)

In [ ]:
# For some reason, this replaces its own file so have to be done this way
# First change the avfrac in the exp_configs 
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Vinitsky_et_al/Trained_policies/60_percent/PPO_BottleneckDesiredVelocityEnv-v0_7e673726_2024-02-26_13-26-01jfvaycan 218\
    --method vinitsky --num_rollouts 1 --gen_emission --shock --render --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### Ours
- Now we have different policies for each av_frac
- At test time, set min_gap to 5

### (5%)

In [ ]:
###### SAFETY + STABILITY ######
# First change the avfrac in the exp_configs 
# For some reason, this replaces its own file so have to be done this way
# Copy the exp configs to the test_rllib.py folder
# Turn on the test time only features in the env
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Ours/Trained_policies/5_percent/\
    --method ours --num_rollouts 1 --gen_emission --shock --render --min_gap 10.0 --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220 


In [ ]:
!python eval_metrics.py --method ours --start_time 500 --end_time 1220 --sim_step 0.5

In [8]:
###### Efficiency ######
# First change the avfrac in the exp_configs 
# For some reason, this replaces its own file so have to be done this way
# Copy the exp configs to the test_rllib.py folder
# Turn on the test time only features in the env
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Ours/Trained_policies/5_percent/PPO_DensityAwareBottleneckEnv-v1_4abee4a6_2024-03-05_15-09-37pxo02uz5 435\
    --method ours --num_rollouts 1 --gen_emission --shock --render --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220 
    

2024-03-05 18:45:14,227	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2024-03-05 18:45:14,228	INFO resource_spec.py:216 -- Starting Ray with 26.17 GiB memory available for workers and up to 13.09 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
/mnt/c/Users/09_gi/Desktop/Beyond-Simulated-Drivers/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute minGap in SumoCarFollowingParams is deprecated, use min_gap instead.
  PendingDeprecationWarning
/mnt/c/Users/09_gi/Desktop/Beyond-Simulated-Drivers/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/mnt/c/Users/09_gi/Desktop/Beyond-Simulated-Drivers/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instea

Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow spe

Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.159844884327468
Free flow speed: 3.1

Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.920391220

Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.9203912203087485
Free flow speed: 3.920391220

...

Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow speed: 0.0
Free flow spe

Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.5352550611815103
Free flow speed: 2.535255061

Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.0

Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.0

Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.019437095702453
Free flow speed: 4.0

In [ ]:
!python eval_metrics.py --method ours --start_time 500 --end_time 1220 --sim_step 0.5

Calculating metrics for 1 rollouts on files: 
['./test_time_rollout/ours/bottleneck_ours_20240305-1845161709682316.1164594-0_emission.csv', './test_time_rollout/ours/bottleneck_ours_20240305-1846391709682399.760109-0_emission.csv', './test_time_rollout/ours/bottleneck_ours_20240305-1848081709682488.039578-0_emission.csv', './test_time_rollout/ours/bottleneck_ours_20240305-1849281709682568.3566103-0_emission.csv', './test_time_rollout/ours/bottleneck_ours_20240305-1850501709682650.886822-0_emission.csv']

file: ./test_time_rollout/ours/bottleneck_ours_20240305-1845161709682316.1164594-0_emission.csv
####################
####################


### (20%)

In [ ]:
###### SAFETY + STABILITY ######
# First change the avfrac in the exp_configs 
# For some reason, this replaces its own file so have to be done this way
# Copy the exp configs to the test_rllib.py folder
# Turn on the test time only features in the env

for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Ours/Trained_policies/20_percent/PPO_DensityAwareBottleneckEnv-v1_17fda320_2024-02-21_20-45-44owaz8svt 124\
    --method ours --num_rollouts 1 --gen_emission --shock --render --min_gap 10.0 --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220 
    

In [ ]:
!python eval_metrics.py --method ours --start_time 500 --end_time 1220 --sim_step 0.5

In [ ]:
###### Efficiency ######


In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (40%)

In [ ]:
###### SAFETY + STABILITY ######
# First change the avfrac in the exp_configs 
# For some reason, this replaces its own file so have to be done this way
# Copy the exp configs to the test_rllib.py folder
# Turn on the test time only features in the env
for _ in range(NUM_ROLLOUTS):
    !python test_rllib.py ./Ours/Trained_policies/40_percent/PPO_DensityAwareBottleneckEnv-v1_6ae1fa7c_2024-02-22_07-39-27j_x7njfv 112\
    --method ours --num_rollouts 1 --gen_emission --shock --render --min_gap 10.0 --sim_step 0.5 --shock_start_time 500 \
    --shock_end_time 1220 --horizon 1220 
    

In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

In [ ]:
###### Efficiency ######


In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

### (60%)

In [ ]:
###### SAFETY + STABILITY ######


In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5

In [ ]:
###### Efficiency ######


In [ ]:
!python eval_metrics.py --method vinitsky --start_time 500 --end_time 1220 --sim_step 0.5